In [1]:
using Revise
using ConvexHullPricing
using DataFrames
using Plots
using JLD2
using ProgressBars, JuMP, Gurobi
const UT = ConvexHullPricing.Utilitaries
const OPT = ConvexHullPricing.Optimizer
function mon(array)
    result = Float64[]
    last = array[1]
    for elt in array
      new_last = minimum([elt, last])
      push!(result, new_last)
      last = new_last
    end
    return result
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-02
Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-02


mon (generic function with 1 method)

In [3]:
BEinstances = []
for file in readdir("..//data//belgian"; join=true)
  push!(BEinstances, UT.load_data(file))
end
instance = BEinstances[8]

ConvexHullPricing.Utilitaries.Instance(3000.0, [10139.0, 9933.0, 9830.0, 9549.0, 9496.0, 9442.0, 9345.0, 9251.0, 9098.0, 9082.0  …  10388.0, 10330.0, 10306.0, 10469.0, 10511.0, 10573.0, 10658.0, 10582.0, 10337.0, 10126.0], ConvexHullPricing.Utilitaries.ThermalGen(Any[525.5, 192.5, 14.5, 504, 253, 211.8, 48.7, 48.7, 48.7, 48.7  …  522, 503, 216.3, 216.3, 43, 20.6, 20.6, 16.8, 15.4, 15.4], Any[1051, 350, 14.5, 1008, 460, 385, 203, 203, 203, 203  …  1044, 1006, 432.5, 432.5, 43, 258, 258, 70, 64, 64], Any[35.03333, 10.5, 0, 33.6, 13.8, 11.54667, 10.15, 10.15, 10.15, 10.15  …  34.8, 33.53333, 14.41333, 14.41333, 0, 2.58, 2.58, 3.5, 3.2, 3.2], Any[35.03333, 10.5, 0, 33.6, 13.8, 11.54667, 10.15, 10.15, 10.15, 10.15  …  34.8, 33.53333, 14.41333, 14.41333, 0, 2.58, 2.58, 3.5, 3.2, 3.2], Any[525.5, 192.5, 14.5, 504, 253, 211.8, 48.7, 48.7, 48.7, 48.7  …  522, 503, 216.3, 216.3, 43, 20.6, 20.6, 16.8, 15.4, 15.4], Any[525.5, 192.5, 14.5, 504, 253, 211.8, 48.7, 48.7, 48.7, 48.7  …  522, 503, 216.3

In [ ]:
ThermalGen = ConvexHullPricing.Utilitaries.ThermalGen(
    MinRunCapacity = [6],
    MaxRunCapacity = [16],
    RampUp = [5],
    RampDown = [5],
    StartUp = [6],
    ShutDown = [6],
    UpTime = [1],
    DownTime = [1],
    NoLoadConsumption = [10],
    MarginalCost = [53],
    FixedCost = [30],
)
instance = ConvexHullPricing.Utilitaries.Instance(
    LostLoad = 3000,
    Load = [6 11 16 11],
    ThermalGen = ThermalGen
)
X0 = UT.LP_Relaxation(instance)

In [4]:
XCRG, ITCRG, CRG, TCRG = OPT.tCRG(instance, 1., 1e-5, 1)

In [ ]:
plot(TCG[2:end], (FS .- FCG), label = "CG")
plot!(TCRG[2:end], (FS .- FCRG), label = "CRG")

In [ ]:
FCG = Float64[]
for price in ITCG
    push!(FCG, UT.exact_oracle(instance, price)[1])
end